In [42]:
import numpy as np
import random
from copy import deepcopy

class SimpleSudokuGame:
    def __init__(self):
        self.size = 9
        self.grid = np.zeros((9, 9), dtype=int)
        self.solution = np.zeros((9, 9), dtype=int)
        self.initial_grid = np.zeros((9, 9), dtype=int)
        
    def is_valid_move(self, grid, row, col, num):
        """Check if placing num at (row, col) is valid"""
        # Check row
        if num in grid[row]:
            return False
        
        # Check column
        if num in grid[:, col]:
            return False
        
        # Check 3x3 box
        box_row, box_col = 3 * (row // 3), 3 * (col // 3)
        if num in grid[box_row:box_row+3, box_col:box_col+3]:
            return False
        
        return True
    
    def solve_sudoku(self, grid):
        """Solve sudoku using backtracking"""
        for row in range(9):
            for col in range(9):
                if grid[row][col] == 0:
                    for num in range(1, 10):
                        if self.is_valid_move(grid, row, col, num):
                            grid[row][col] = num
                            if self.solve_sudoku(grid):
                                return True
                            grid[row][col] = 0
                    return False
        return True
    
    def count_solutions(self, grid, max_solutions=2):
        """Count number of solutions (stops at max_solutions for efficiency)"""
        solutions = []
        
        def backtrack(g):
            if len(solutions) >= max_solutions:
                return
            
            for row in range(9):
                for col in range(9):
                    if g[row][col] == 0:
                        for num in range(1, 10):
                            if self.is_valid_move(g, row, col, num):
                                g[row][col] = num
                                backtrack(g)
                                g[row][col] = 0
                        return
            
            solutions.append(deepcopy(g))
        
        backtrack(deepcopy(grid))
        return len(solutions)
    
    def generate_complete_sudoku(self):
        """Generate a complete valid sudoku solution"""
        grid = np.zeros((9, 9), dtype=int)
        
        def fill_grid(grid):
            for row in range(9):
                for col in range(9):
                    if grid[row][col] == 0:
                        numbers = list(range(1, 10))
                        random.shuffle(numbers)
                        
                        for num in numbers:
                            if self.is_valid_move(grid, row, col, num):
                                grid[row][col] = num
                                if fill_grid(grid):
                                    return True
                                grid[row][col] = 0
                        return False
            return True
        
        fill_grid(grid)
        return grid
    
    def shuffle_grid(self, grid, iterations=50):
        """Shuffle the grid while maintaining validity"""
        shuffled = deepcopy(grid)
        
        for _ in range(iterations):
            operation = random.choice(['swap_rows', 'swap_cols'])
            
            if operation == 'swap_rows':
                block = random.randint(0, 2)
                row1 = block * 3 + random.randint(0, 2)
                row2 = block * 3 + random.randint(0, 2)
                shuffled[[row1, row2]] = shuffled[[row2, row1]]
                
            elif operation == 'swap_cols':
                block = random.randint(0, 2)
                col1 = block * 3 + random.randint(0, 2)
                col2 = block * 3 + random.randint(0, 2)
                shuffled[:, [col1, col2]] = shuffled[:, [col2, col1]]
        
        return shuffled
    
    def create_puzzle(self, difficulty='medium'):
        """Create a puzzle by removing numbers from a complete grid"""
        clue_counts = {
            'easy': random.randint(40, 45),
            'medium': random.randint(30, 35),
            'hard': random.randint(25, 30)
        }
        
        target_clues = clue_counts[difficulty]
        print(f"🔄 Generating {difficulty} puzzle with ~{target_clues} clues...")
        
        # Generate and shuffle a complete solution
        complete_grid = self.generate_complete_sudoku()
        shuffled_grid = self.shuffle_grid(complete_grid)
        self.solution = deepcopy(shuffled_grid)
        
        # Create list of all positions and shuffle
        positions = [(i, j) for i in range(9) for j in range(9)]
        random.shuffle(positions)
        
        puzzle = deepcopy(shuffled_grid)
        clues_remaining = 81
        
        # Remove numbers while maintaining unique solution
        for row, col in positions:
            if clues_remaining <= target_clues:
                break
                
            original_value = puzzle[row][col]
            puzzle[row][col] = 0
            
            if self.count_solutions(puzzle) == 1:
                clues_remaining -= 1
            else:
                puzzle[row][col] = original_value
        
        self.grid = puzzle
        self.initial_grid = deepcopy(puzzle)
        print(f"✅ Puzzle created with {clues_remaining} clues!")
        
        return puzzle
    
    def print_grid(self):
        """Print the grid in a nice format"""
        print("\n" + "=" * 37)
        print("    1 2 3   4 5 6   7 8 9")
        print("  ┌───────┬───────┬───────┐")
        
        for i in range(9):
            if i > 0 and i % 3 == 0:
                print("  ├───────┼───────┼───────┤")
            
            row_str = f"{i+1} │"
            for j in range(9):
                if j > 0 and j % 3 == 0:
                    row_str += "│"
                
                if self.grid[i, j] == 0:
                    row_str += " ."
                else:
                    # Show initial numbers in one way, user numbers in another
                    if self.initial_grid[i, j] != 0:
                        row_str += f" {self.grid[i, j]}"  # Initial clues
                    else:
                        row_str += f" \033[94m{self.grid[i, j]}\033[0m"  # User moves in blue
                
                row_str += " "
            
            row_str += "│"
            print(row_str)
        
        print("  └───────┴───────┴───────┘")
        print("=" * 37 + "\n")
    
    def make_move(self, row, col, num):
        """Make a move if valid"""
        # Convert to 0-indexed
        row, col = row - 1, col - 1
        
        if not (0 <= row <= 8 and 0 <= col <= 8):
            return False, "Invalid position! Use row/col 1-9"
        
        if self.initial_grid[row, col] != 0:
            return False, "Cannot modify initial clue!"
        
        if num == 0:  # Clear the cell
            self.grid[row, col] = 0
            return True, "Cell cleared!"
        
        if not (1 <= num <= 9):
            return False, "Number must be 1-9!"
        
        if self.is_valid_move(self.grid, row, col, num):
            self.grid[row, col] = num
            return True, f"Placed {num} at ({row+1},{col+1})"
        else:
            return False, "Invalid move! Number conflicts with Sudoku rules"
    
    def is_complete(self):
        """Check if puzzle is solved"""
        return np.all(self.grid != 0) and self.is_valid_complete_grid(self.grid)
    
    def is_valid_complete_grid(self, grid):
        """Check if a complete grid is valid"""
        for row in range(9):
            if len(set(grid[row])) != 9:
                return False
        
        for col in range(9):
            if len(set(grid[:, col])) != 9:
                return False
        
        for box_row in range(0, 9, 3):
            for box_col in range(0, 9, 3):
                box = grid[box_row:box_row+3, box_col:box_col+3].flatten()
                if len(set(box)) != 9:
                    return False
        
        return True
    
    def get_hint(self):
        """Get a hint"""
        empty_cells = [(i, j) for i in range(9) for j in range(9) 
                      if self.grid[i, j] == 0]
        
        if empty_cells:
            row, col = random.choice(empty_cells)
            self.grid[row, col] = self.solution[row, col]
            return f"💡 Hint: {self.solution[row, col]} at position ({row+1},{col+1})"
        return "No more hints available!"
    
    def show_solution(self):
        """Reveal the complete solution"""
        self.grid = deepcopy(self.solution)
        print("🎯 Complete solution revealed!")

def play_text_sudoku(difficulty='medium'):
    """Play Sudoku with text interface"""
    print("🎮 Welcome to Text-Based Sudoku!")
    print("=" * 50)
    
    game = SimpleSudokuGame()
    game.create_puzzle(difficulty)
    
    print("\n📋 How to play:")
    print("• Type: move(row, col, number) - e.g., move(1, 5, 7)")
    print("• Type: move(row, col, 0) - to clear a cell")
    print("• Type: hint() - get a hint")
    print("• Type: solution() - see complete solution")
    print("• Type: new_easy(), new_medium(), new_hard() - start new game")
    print("• Type: check() - check if solved")
    
    # Show initial puzzle
    game.print_grid()
    
    # Define helper functions that can be called
    def move(row, col, num):
        success, message = game.make_move(row, col, num)
        print(f"➤ {message}")
        if success:
            game.print_grid()
            if game.is_complete():
                print("🎉 CONGRATULATIONS! PUZZLE SOLVED! 🎉")
        return success
    
    def hint():
        message = game.get_hint()
        print(f"➤ {message}")
        game.print_grid()
        if game.is_complete():
            print("🎉 CONGRATULATIONS! PUZZLE SOLVED! 🎉")
    
    def solution():
        game.show_solution()
        game.print_grid()
    
    def check():
        if game.is_complete():
            print("🎉 Perfect! Puzzle solved correctly!")
        elif np.array_equal(game.grid, game.initial_grid):
            print("🤔 You haven't made any moves yet!")
        else:
            empty_count = np.sum(game.grid == 0)
            print(f"🔍 Keep going! {empty_count} cells remaining.")
    
    def new_easy():
        return play_text_sudoku('easy')
    
    def new_medium():
        return play_text_sudoku('medium')
        
    def new_hard():
        return play_text_sudoku('hard')
    
    # Return the game object and helper functions
    return {
        'game': game,
        'move': move,
        'hint': hint,
        'solution': solution,
        'check': check,
        'new_easy': new_easy,
        'new_medium': new_medium,
        'new_hard': new_hard
    }

# Quick start functions
def start_easy():
    return play_text_sudoku('easy')

def start_medium():
    return play_text_sudoku('medium')
    
def start_hard():
    return play_text_sudoku('hard')

In [43]:
sudoku=start_easy()

🎮 Welcome to Text-Based Sudoku!
🔄 Generating easy puzzle with ~43 clues...
✅ Puzzle created with 43 clues!

📋 How to play:
• Type: move(row, col, number) - e.g., move(1, 5, 7)
• Type: move(row, col, 0) - to clear a cell
• Type: hint() - get a hint
• Type: solution() - see complete solution
• Type: new_easy(), new_medium(), new_hard() - start new game
• Type: check() - check if solved

    1 2 3   4 5 6   7 8 9
  ┌───────┬───────┬───────┐
1 │ 9  8  3 │ 7  .  . │ .  1  2 │
2 │ 1  4  6 │ 3  2  . │ 7  .  . │
3 │ 5  7  2 │ .  .  . │ 4  .  . │
  ├───────┼───────┼───────┤
4 │ .  .  . │ 1  9  4 │ .  .  5 │
5 │ 7  .  . │ 6  .  . │ .  .  9 │
6 │ .  5  9 │ .  7  . │ .  6  . │
  ├───────┼───────┼───────┤
7 │ 8  .  5 │ 4  1  . │ 9  3  . │
8 │ 4  .  . │ 5  .  . │ .  8  1 │
9 │ 6  .  1 │ 9  .  2 │ 5  .  7 │
  └───────┴───────┴───────┘



In [44]:
sudoku['move'](1, 2, 7)

➤ Cannot modify initial clue!


False

In [45]:
sudoku['hint']()

➤ 💡 Hint: 3 at position (3,9)

    1 2 3   4 5 6   7 8 9
  ┌───────┬───────┬───────┐
1 │ 9  8  3 │ 7  .  . │ .  1  2 │
2 │ 1  4  6 │ 3  2  . │ 7  .  . │
3 │ 5  7  2 │ .  .  . │ 4  .  3 │
  ├───────┼───────┼───────┤
4 │ .  .  . │ 1  9  4 │ .  .  5 │
5 │ 7  .  . │ 6  .  . │ .  .  9 │
6 │ .  5  9 │ .  7  . │ .  6  . │
  ├───────┼───────┼───────┤
7 │ 8  .  5 │ 4  1  . │ 9  3  . │
8 │ 4  .  . │ 5  .  . │ .  8  1 │
9 │ 6  .  1 │ 9  .  2 │ 5  .  7 │
  └───────┴───────┴───────┘



In [46]:
sudoku['check']()

🔍 Keep going! 37 cells remaining.


In [47]:
sudoku['solution']()

🎯 Complete solution revealed!

    1 2 3   4 5 6   7 8 9
  ┌───────┬───────┬───────┐
1 │ 9  8  3 │ 7  4  5 │ 6  1  2 │
2 │ 1  4  6 │ 3  2  9 │ 7  5  8 │
3 │ 5  7  2 │ 8  6  1 │ 4  9  3 │
  ├───────┼───────┼───────┤
4 │ 2  6  8 │ 1  9  4 │ 3  7  5 │
5 │ 7  1  4 │ 6  5  3 │ 8  2  9 │
6 │ 3  5  9 │ 2  7  8 │ 1  6  4 │
  ├───────┼───────┼───────┤
7 │ 8  2  5 │ 4  1  7 │ 9  3  6 │
8 │ 4  9  7 │ 5  3  6 │ 2  8  1 │
9 │ 6  3  1 │ 9  8  2 │ 5  4  7 │
  └───────┴───────┴───────┘



In [48]:
!pip show ipywidgets


Name: ipywidgets
Version: 7.6.5
Summary: IPython HTML widgets for Jupyter
Home-page: http://ipython.org
Author: IPython Development Team
Author-email: ipython-dev@scipy.org
License: BSD
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: ipykernel, ipython, ipython-genutils, jupyterlab-widgets, nbformat, traitlets, widgetsnbextension
Required-by: jupyter


In [60]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def display_board(game, update_callback):
    grid = widgets.GridBox(
        children=[],
        layout=widgets.Layout(
            width='100%',
            grid_template_columns='repeat(9, 42px)',
            grid_template_rows='repeat(9, 42px)',
            grid_gap='0px'
        )
    )

    buttons = []

    for i in range(9):
        row = []
        for j in range(9):
            num = game.grid[i][j]
            is_fixed = game.initial_grid[i][j] != 0

            if num == 0:
                label = ""
            else:
                label = str(num)

            # Márgenes para separar bloques 3x3 visualmente
            margin_top = '4px' if i % 3 == 0 else '1px'
            margin_left = '4px' if j % 3 == 0 else '1px'

            btn = widgets.Button(
                description=label,
                layout=widgets.Layout(
                    width='40px',
                    height='40px',
                    margin=f"{margin_top} 1px 1px {margin_left}"
                ),
                disabled=bool(is_fixed)
            )

            def on_click(btn, i=i, j=j):
                update_callback(i, j)

            btn.on_click(on_click)
            row.append(btn)
            grid.children += (btn,)
        buttons.append(row)

    display(grid)
    return buttons




In [61]:
def interactive_sudoku_ui(game):
    selected_cell = {'row': None, 'col': None}
    
    output = widgets.Output()
    
    def update_cell(i, j):
        selected_cell['row'] = i
        selected_cell['col'] = j
        with output:
            clear_output()
            print(f"Seleccionaste la celda ({i+1}, {j+1}). Escoge un número:")

            number_picker = widgets.BoundedIntText(
                value=1,
                min=0,
                max=9,
                description='Número:',
                layout=widgets.Layout(width='200px')
            )

            def on_submit(change):
                row, col = selected_cell['row'], selected_cell['col']
                success, msg = game.make_move(row+1, col+1, change['new'])
                clear_output()
                print(msg)
                redraw()

            number_picker.observe(on_submit, names='value')
            display(number_picker)

    def redraw():
        display_board(game, update_cell)

    redraw()
    display(output)



In [63]:
game = SimpleSudokuGame()
game.create_puzzle('easy')
interactive_sudoku_ui(game)


Invalid move! Number conflicts with Sudoku rules


GridBox(children=(Button(description='4', disabled=True, layout=Layout(height='40px', margin='4px 1px 1px 4px'…